In [1]:
import os
from dateutil.parser import parse as dateutil_parse
import pandas as pd

PATH = 'season_1/'
CLEAN_PATH = PATH+'clean/'

%matplotlib inline

In [69]:
cluster_map = pd.read_csv(CLEAN_PATH+'cluster_map.csv',index_col=0)
poi = pd.read_csv(CLEAN_PATH+'poi.csv',index_col=0)
train_order = pd.read_pickle(CLEAN_PATH+'train_order.pickle')
test_order = pd.read_pickle(CLEAN_PATH+'test_order.pickle')
train_traffic = pd.read_pickle(CLEAN_PATH+'train_traffic.pickle')
test_traffic = pd.read_pickle(CLEAN_PATH+'test_traffic.pickle')
train_weather = pd.read_pickle(CLEAN_PATH+'train_weather.pickle')
test_weather = pd.read_pickle(CLEAN_PATH+'test_weather.pickle')

In [3]:
train_order_index = pd.Series(range(len(train_order)),index=train_order['Time'])
test_order_index = pd.Series(range(len(test_order)),index=test_order['Time'])

# powerful datetime index

In [4]:
train_order.ix[train_order_index.ix['20160101 13:00:10':'2016-01-01 130012']]

,request,start_district_hash,Price,Time
7731,True,929ec6c160e6f52c20a4217c7978f681,15.5,2016-01-01 13:00:11
19352,True,d4ec2125aff74eded207d2d915ef682f,18.0,2016-01-01 13:00:12
46153,True,ca064c2682ca48c6a21de012e87c0df5,7.8,2016-01-01 13:00:12
60280,True,91690261186ae5bee8f83808ea1e4a01,9.0,2016-01-01 13:00:12
67821,False,929ec6c160e6f52c20a4217c7978f681,14.0,2016-01-01 13:00:12
74052,False,4725c39a5e5f4c188d382da3910b3f3f,11.0,2016-01-01 13:00:12
81666,True,929ec6c160e6f52c20a4217c7978f681,12.0,2016-01-01 13:00:12
81984,True,d4ec2125aff74eded207d2d915ef682f,13.0,2016-01-01 13:00:12
151606,True,74c1c25f4b283fa74a5514307b0d0278,9.0,2016-01-01 13:00:10
177238,True,82cc4851f9e4faa4e54309f8bb73fd7c,5.0,2016-01-01 13:00:11


# deal with time slot

In [38]:
tmp = pd.read_table(PATH+'test_set_1/read_me_1.txt')
test_target = pd.Series(map(lambda x: int(x.split('-')[-1]),tmp.values.T[0]),index = map(lambda x: dateutil_parse(x[:10]), tmp.values.T[0]))
test_target.name = 'timeslot'
test_target.index.name = 'datetime'

In [40]:
test_target.ix['20160122']

datetime
2016-01-22     46
2016-01-22     58
2016-01-22     70
2016-01-22     82
2016-01-22     94
2016-01-22    106
2016-01-22    118
2016-01-22    130
2016-01-22    142
Name: timeslot, dtype: int64

In [53]:
test_target.unique()

array([ 46,  58,  70,  82,  94, 106, 118, 130, 142], dtype=int64)

In [51]:
test_slot = test_order['Time'].map(lambda x: (x.hour*60+x.minute)/10+1)

In [62]:
date = '20160128'
print len(test_target.ix[date])
print len(test_weather.ix[date])

8
13


In [29]:
test_weather.ix['20160122']

,weather,temperature,pm25
datatime,,,
2016-01-22 07:02:10,2,2.0,51
2016-01-22 07:22:35,2,2.0,51
2016-01-22 09:12:48,4,1.0,53
2016-01-22 09:28:39,4,1.0,53
2016-01-22 15:12:31,6,1.0,64
2016-01-22 19:28:07,4,1.0,52
2016-01-22 23:07:35,6,1.0,98
2016-01-22 23:21:04,6,1.0,98


# check what the test set have

In [70]:
test_order['timeslot'] = test_order['Time'].map(lambda x: (x.hour*60+x.minute)/10+1)

In [71]:
test_weather['timeslot'] = test_weather.index.map(lambda x: (x.hour*60+x.minute)/10+1)

In [82]:
sorted(test_target.unique())

[46, 58, 70, 82, 94, 106, 118, 130, 142]

In [98]:
target_need = reduce(lambda x,y: x+y,test_target.ix['20160122'].map(lambda x: [x-3, x-2,x-1]))

In [92]:
test_weather['timeslot'].ix['20160122']

datatime
2016-01-22 07:02:10     43
2016-01-22 07:22:35     45
2016-01-22 09:12:48     56
2016-01-22 09:28:39     57
2016-01-22 15:12:31     92
2016-01-22 19:28:07    117
2016-01-22 23:07:35    139
2016-01-22 23:21:04    141
Name: timeslot, dtype: int32

In [106]:
test_target.ix['20160122'].map(lambda x: [x-3, x-2,x-1])

datetime
2016-01-22       [43, 44, 45]
2016-01-22       [55, 56, 57]
2016-01-22       [67, 68, 69]
2016-01-22       [79, 80, 81]
2016-01-22       [91, 92, 93]
2016-01-22    [103, 104, 105]
2016-01-22    [115, 116, 117]
2016-01-22    [127, 128, 129]
2016-01-22    [139, 140, 141]
Name: timeslot, dtype: object

In [100]:
set(reduce(lambda x,y: x+y,test_target.ix['20160122'].map(lambda x: [x-3, x-2,x-1]))) - set(test_order['timeslot'].ix[test_order_index.ix['20160122']])

set()

In [113]:
date = '20160130'
print 'order',set(reduce(lambda x,y: x+y,test_target.ix[date].map(lambda x: [x-3, x-2,x-1]))) - set(test_order['timeslot'].ix[test_order_index.ix[date]])
print 'order',set(test_order['timeslot'].ix[test_order_index.ix[date]]) - set(reduce(lambda x,y: x+y,test_target.ix[date].map(lambda x: [x-3, x-2,x-1])))
print 'weather',sorted(set(reduce(lambda x,y: x+y,test_target.ix[date].map(lambda x: [x-3, x-2,x-1]))) - set(test_weather['timeslot'].ix[date]))
print 'weather',set(test_weather['timeslot'].ix[date]) - set(reduce(lambda x,y: x+y,test_target.ix[date].map(lambda x: [x-3, x-2,x-1])))

 order set([])
order set([])
weather [104, 128]
weather set([])


In [95]:
test_order

,request,start_district_hash,Price,Time,timeslot
0,False,b05379ac3f9b7d99370d443cfd5dcc28,7.0,2016-01-22 17:19:43,104
1,True,b05379ac3f9b7d99370d443cfd5dcc28,12.0,2016-01-22 17:19:43,104
2,True,62afaf3288e236b389af9cfdc5206415,18.0,2016-01-22 11:00:05,67
3,False,d4ec2125aff74eded207d2d915ef682f,36.0,2016-01-22 17:00:42,103
4,False,d4ec2125aff74eded207d2d915ef682f,36.0,2016-01-22 17:00:42,103
5,True,82cc4851f9e4faa4e54309f8bb73fd7c,12.0,2016-01-22 21:00:29,127
6,True,82cc4851f9e4faa4e54309f8bb73fd7c,17.5,2016-01-22 17:29:25,105
7,True,b05379ac3f9b7d99370d443cfd5dcc28,11.8,2016-01-22 09:14:30,56
8,False,b05379ac3f9b7d99370d443cfd5dcc28,10.0,2016-01-22 09:14:30,56
9,True,b05379ac3f9b7d99370d443cfd5dcc28,14.0,2016-01-22 09:14:30,56


In [80]:
sorted(test_order['timeslot'].unique())

[43,
 44,
 45,
 55,
 56,
 57,
 67,
 68,
 69,
 79,
 80,
 81,
 91,
 92,
 93,
 103,
 104,
 105,
 115,
 116,
 117,
 127,
 128,
 129,
 139,
 140,
 141]

In [87]:
set(test_weather['timeslot'].unique())-set(test_order['timeslot'].unique())

set()

In [108]:
test_target.ix['20160124']

datetime
2016-01-24     58
2016-01-24     70
2016-01-24     82
2016-01-24     94
2016-01-24    106
2016-01-24    118
2016-01-24    130
2016-01-24    142
Name: timeslot, dtype: int64

In [137]:
test_target

datetime
2016-01-22     46
2016-01-22     58
2016-01-22     70
2016-01-22     82
2016-01-22     94
2016-01-22    106
2016-01-22    118
2016-01-22    130
2016-01-22    142
2016-01-24     58
2016-01-24     70
2016-01-24     82
2016-01-24     94
2016-01-24    106
2016-01-24    118
2016-01-24    130
2016-01-24    142
2016-01-26     46
2016-01-26     58
2016-01-26     70
2016-01-26     82
2016-01-26     94
2016-01-26    106
2016-01-26    118
2016-01-26    130
2016-01-26    142
2016-01-28     58
2016-01-28     70
2016-01-28     82
2016-01-28     94
2016-01-28    106
2016-01-28    118
2016-01-28    130
2016-01-28    142
2016-01-30     46
2016-01-30     58
2016-01-30     70
2016-01-30     82
2016-01-30     94
2016-01-30    106
2016-01-30    118
2016-01-30    130
2016-01-30    142
Name: timeslot, dtype: int64